In [ ]:
import kenlm
import nltk
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from string import punctuation
from collections import defaultdict
import random

## Generate Candidate Words

In [ ]:
def generate_one_edit_distance(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def generate_two_edit_distance(word):
    return set(e2 for e1 in generate_one_edit_distance(word) for e2 in generate_one_edit_distance(e1))

## Unigram

Our calculation for unigram does not use any smoothing, since we argue the exact probabilities of each words does not matter (we only care about which words is more probable than others). We do not use the KenLM for unigram since the author only implement it for bigram and higher order.

In [ ]:
path = "./ind_news_2022_1M-sentences.txt"
data = []
with open(path, "r", encoding="UTF-8") as file:
    i = 0
    for line in file:
        # print("Iteration", str(i), ": ", line)
        i = i + 1;
        data.append(line)

In [ ]:
print(word_tokenize(data[0]))

In [ ]:
def preprocess_text(sentence):
  words = word_tokenize(sentence)
  processed_words = []
  for word in words:
    if word.isnumeric():
      processed_words.append("n4m3r1c")
    elif word not in punctuation and word != "''" and word != "“" and word != "”":
      processed_words.append(word.lower())
  return processed_words

In [ ]:
unigram_freq = dict()

for line in data:
	list_words = preprocess_text(line)
	for word in list_words:
		if word in unigram_freq.keys():
			unigram_freq[word] += 1
		else:
			unigram_freq[word] = 1

In [ ]:
def get_sorted_unigram(word_list):
	sorted_list = []
	for word in word_list:
		if word in unigram_freq.keys():
			sorted_list.append((word, unigram_freq[word]))
		else:
			sorted_list.append((word, 0))
	sorted_list.sort(key=lambda x: x[1], reverse=True)
	return sorted_list

## Bigram and Trigram Model

We use KenLM to help us to compute the bigram and trigram probabilities with Knesser-Ney smoothing. We already train the model using 1 million sentence from Indonesian News.

In [ ]:
class bigram_sorter:
  def __init__(self):
    self.model_bigram = kenlm.Model("./cleaned_corpus_bigram.klm")

  def get_sorted_list(self, preceeding_sentence, word_list, following_sentence):
    sorted_list = []
    for word in word_list:
      new_sentence = preceeding_sentence+" "+word+" "+following_sentence
      sorted_list.append((word, self.model_bigram.score(new_sentence)))
    sorted_list.sort(key=lambda x: x[1], reverse=True)
    return sorted_list


class trigram_sorter:
  def __init__(self):
    self.model_trigram = kenlm.Model("./cleaned_corpus_trigram.klm")

  def get_sorted_list(self, preceeding_sentence, word_list, following_sentence):
    sorted_list = []
    for word in word_list:
      new_sentence = preceeding_sentence+" "+word+" "+following_sentence
      sorted_list.append((word, self.model_trigram.score(new_sentence)))
    sorted_list.sort(key=lambda x: x[1], reverse=True)
    return sorted_list


In [ ]:
sorted_list = bigram_sorter()
sorted_list.get_sorted_list("aku", ["pntar", "pintar", "pantir", "petir"])

In [ ]:
path = "./list_kbbi.txt"
kbbi_words = set()
with open(path, "r") as file:
    for i, line in enumerate(file):
        # print("Iteration", str(i), ": ", line)
        words = line.rstrip('\n').split(' ')
        for word in words:
          if(word != "" and word.isalpha()):
            kbbi_words.add(str(word))

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
sentence = "n4m3r1c hai"
words = word_tokenize(sentence)
print(words)

## Spell Corrector Class

In [ ]:
class Spell_Corrector:
	def __init__(self):
		self.bigram = bigram_sorter()
		self.trigram = trigram_sorter()

	def generate_correction(self, sentence):
		sentence = sentence.lower()
		words = word_tokenize(sentence)

		list_unigram = []
		list_bigram = []
		list_trigram = []

		for index, word in enumerate(words):
			stemmed_word = stemmer.stem(word)
			# check if this word is non-word error
			if stemmed_word not in kbbi_words and word != "n4m3r1c" and word not in punctuation:
				# not exists in KBBI
				possible_corrections = generate_one_edit_distance(word)
				possible_corrections = possible_corrections.union(generate_two_edit_distance(word))
				filtered_corrections = []
				
				# filter possible word corrections using KBBI
				for possible_word in possible_corrections:
					if possible_word in kbbi_words:
						filtered_corrections.append(possible_word)

				# sort the list based on the N-gram probability
				
				# Use unigram
				sorted_list_unigram = get_sorted_unigram(filtered_corrections)
				list_unigram.append(sorted_list_unigram[:5])

				# Use bigram
				sorted_list_bigram = self.bigram.get_sorted_list(
					(words[index-1] if index > 0 else ""), 
					filtered_corrections, 
					(words[index+1] if index < len(words)-1 else ""))
				list_bigram.append(sorted_list_bigram[:5])

				# Use trigram
				sorted_list_trigram = self.trigram.get_sorted_list(
					(words[index-2] if index > 1 else "") + (words[index-1] if index > 0 else ""), 
					filtered_corrections,
					(words[index+2] if index < len(words)-2 else "") + (words[index+1] if index < len(words)-1 else ""))
				list_trigram.append(sorted_list_trigram[:5])
			
			else:
				list_unigram.append([])
				list_bigram.append([])
				list_trigram.append([])

		return list_trigram
			

	def print_correction(self, sentence):
		unigram_corr, bigram_corr, trigram_corr = self.generate_correction(sentence)
		sentence = sentence.lower()
		words = word_tokenize(sentence)
		for i, word in enumerate(words):
			print(f"Correction for {word}")

			print("Unigram: ")
			print(unigram_corr[i])

			print("Bigram: ")
			print(bigram_corr[i])

			print("Trigram: ")
			print(trigram_corr[i])

In [ ]:
id_spell_checker = Spell_Corrector()
id_spell_checker.print_correction("indonesia memiliki sumbr daya alam yang mllimpah")

## Testing

In [ ]:
testing_path = "./cleaned_test.txt"
data = []
with open(testing_path, "r", encoding="UTF-8") as file:
    i = 0
    for line in file:
        # print("Iteration", str(i), ": ", line)
        i = i + 1;
        data.append(line)

In [ ]:
print(data[0])

Generating Typo

In [ ]:
P = 0.3
legit_sents = []
typo_sents = []
for sentence in data:
	words = sentence.split(" ")
	legit_words = []
	typo_words = []
	for word in words:
		rand = random.random()
		if word == "" or word == "\n" or word == " ":
			continue
		legit_words.append(word)
		if rand <= P and word != "n4m3r1c":
			edit_distance = random.randint(1, 2)

			if edit_distance == 1:
				word_cand = list(generate_one_edit_distance(word))
				typo_words.append(word_cand[random.randint(0, len(word_cand)-1)])
			else:
				word_cand1 = list(generate_one_edit_distance(word))
				word_cand2 = list(generate_one_edit_distance(word_cand1[random.randint(0, len(word_cand1)-1)]))
				typo_words.append(word_cand2[random.randint(0, len(word_cand2)-1)])
		else:
			typo_words.append(word)
	new_sentence = ""
	for word in typo_words:
		new_sentence += word + " "
	typo_sents.append(new_sentence)

	new_sentence = ""
	for word in legit_words:
		new_sentence += word + " "
	legit_sents.append(new_sentence)

In [ ]:
f = open("typo_test.txt", "w", encoding="UTF-8")
for sentence in new_data:
	f.write(sentence + "\n")
f.close()

In [ ]:
total_typo = 0
# accuracy for 3, 4, and 5 top list
unigram_acc = [0, 0, 0]
bigram_acc = [0, 0, 0]
trigram_acc = [0, 0, 0]

spell_corrector = Spell_Corrector()

cnt = 1

for (real_sentence, typo_sentence) in zip(legit_sents, typo_sents):
	# print(real_sentence)
	# print(typo_sentence)

	cnt += 1

	if cnt % 500 == 0:
		print(cnt)

	real_words = word_tokenize(real_sentence)
	typo_words = word_tokenize(typo_sentence)
	unigram_lists, bigram_lists, trigram_lists = spell_corrector.generate_correction(typo_sentence)
	for idx, (real_word, typo_word, unigram_list, bigram_list, trigram_list) in enumerate(zip(real_words, typo_words, unigram_lists, bigram_lists trigram_lists)):
		if real_word == typo_word:
			continue

		unigram_list = [uni[0] for uni in unigram_list]
		bigram_list = [bi[0] for bi in bigram_list]
		trigram_list = [tri[0] for tri in trigram_list]


		total_typo += 1

		if real_word in unigram_list[:min(3, len(unigram_list))]:
			unigram_acc[0] += 1
		
		if real_word in unigram_list[:min(4, len(unigram_list))]:
			unigram_acc[1] += 1
		
		if real_word in unigram_list[:min(5, len(unigram_list))]:
			unigram_acc[2] += 1

		if real_word in bigram_list[:min(3, len(bigram_list))]:
			bigram_acc[0] += 1
		
		if real_word in bigram_list[:min(4, len(bigram_list))]:
			bigram_acc[1] += 1
		
		if real_word in bigram_list[:min(5, len(bigram_list))]:
			bigram_acc[2] += 1
		
		if real_word in trigram_list[:min(3, len(trigram_list))]:
			trigram_acc[0] += 1
		
		if real_word in trigram_list[:min(4, len(trigram_list))]:
			trigram_acc[1] += 1
		
		if real_word in trigram_list[:min(5, len(trigram_list))]:
			trigram_acc[2] += 1


In [ ]:
print(total_typo)
for i in range(3):
	print(f"Unigram Top {i+3} List: {unigram_acc[i]/total_typo}")

for i in range(3):
	print(f"Bigram Top {i+3} List: {bigram_acc[i]/total_typo}")	

for i in range(3):
	print(f"Trigram Top {i+3} List: {trigram_acc[i]/total_typo}")

print((unigram_acc[0]+unigram_acc[1]+unigram_acc[2])/(3*total_typo))
print((bigram_acc[0]+bigram_acc[1]+bigram_acc[2])/(3*total_typo))
print((trigram_acc[0]+trigram_acc[1]+trigram_acc[2])/(3*total_typo))